In [ ]:
from transformers import pipeline

llm = pipeline("text-generation", model="gpt2")
prompt = "Q: What is a loan? A: A loan is borrowed money repaid with interest. Q: What is KYC? A:"
response = llm(prompt, max_length=100)[0]["generated_text"]
print("Response:", response)  # Know Your Customer...

In [ ]:
# Requires OpenAI API key
import openai

openai.api_key = "your-api-key"
prompt = "Q: What is a loan? A: A loan is borrowed money repaid with interest. Q: What is KYC? A:"
response = openai.Completion.create(model="gpt-3.5-turbo", prompt=prompt, max_tokens=50)
print("Response:", response.choices[0].text)  # Know Your Customer...

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
text = "HDFC Bank offers low-interest loan"
tokens = tokenizer.tokenize(text)
token_ids = tokenizer(text)['input_ids']
print("Tokens:", tokens)  # ['HD', 'FC', 'Bank', 'offers', 'low', '-', 'interest', 'loan']
print("Token IDs:", token_ids)  # [e.g., 1234, 5678, ...]
print("Token Count:", len(tokens))  # 8 tokens

#### Embeddings and Parameters

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
text = "HDFC Bank offers loan"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
embeddings = outputs.last_hidden_state  # [1, seq_len, 768]
print("BERT Embeddings Shape:", embeddings.shape)  # [1, 6, 768]

In [ ]:
# Requires OpenAI API key
import openai

openai.api_key = "your-api-key"  # Replace with actual key
text = "HDFC Bank offers loan"
response = openai.Embedding.create(input=text, model="text-embedding-ada-002")
embedding = response['data'][0]['embedding']
print("OpenAI Embedding Length:", len(embedding))  # 1536

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
text = "HDFC Bank offers loan"
embedding = model.encode(text)
print("Sentence-BERT Embedding Shape:", embedding.shape)  # [384]

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
inputs = tokenizer("How to reset password?", return_tensors="pt")
# Low temperature
#outputs_low = model.generate(**inputs, max_length=50, temperature=0.2)
#print("Low Temp:", tokenizer.decode(outputs_low[0]))
# High temperature
outputs_high = model.generate(**inputs, max_length=50, temperature=1)
print("High Temp:", tokenizer.decode(outputs_high[0]))

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

def generate_text(prompt, temperature):
    # Load pre-trained GPT-2 model and tokenizer
    model_name = 'gpt2'
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    # Encode prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Generate text
    output = model.generate(
        input_ids,
        max_length=100,
        temperature=temperature,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and return
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Prompt
prompt = "Once upon a time in a distant galaxy,"

# Low temperature (more deterministic)
low_temp_output = generate_text(prompt, temperature=0.3)

# High temperature (more creative/random)
high_temp_output = generate_text(prompt, temperature=1.0)

# Display results
print("\n--- Low Temperature Output (0.3) ---\n")
print(low_temp_output)

print("\n--- High Temperature Output (1.0) ---\n")
print(high_temp_output)


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
inputs = tokenizer("HDFC Bank offers loan", return_tensors="pt")
outputs = model(**inputs)
print("BERT Sentiment:", torch.softmax(outputs.logits, dim=-1))

In [ ]:
# Requires OpenAI API key
import openai

openai.api_key = "your-api-key"
response = openai.Completion.create(
    model="gpt-3.5-turbo",
    prompt="Generate a response for: HDFC Bank offers loan",
    max_tokens=50,
    temperature=0.7
)
print("GPT-3.5 Response:", response.choices[0].text)

In [ ]:
# Requires OpenAI API key
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS

llm = ChatOpenAI(model="gpt-3.5-turbo", api_key="your-api-key")
vector_store = FAISS.from_texts(["Diabetes: manage with insulin"], OpenAIEmbeddings(api_key="your-api-key"))
retriever = vector_store.as_retriever()
prompt = f"Context: {{context}}\nQuestion: {{question}}"
context = vector_store.similarity_search("diabetes")[0].page_content
response = llm.invoke(prompt.format(context=context, question="How to manage diabetes?"))
print("Response:", response.content)  # Insulin...

In [ ]:
# Install LangChain (for RAG), PyPDF2 (to read PDFs), FAISS (to search text), and the updated langchain-ollama package
#!conda install -c conda-forge langchain langchain-community pypdf2 faiss-cpu -y
#!pip install -U langchain-ollama

In [ ]:
#!ollama serve

In [ ]:
# Import the tools we’ll use
import PyPDF2
from langchain_community.llms import Ollama
from langchain_ollama import OllamaEmbeddings  # Updated import to fix deprecation
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
import os

In [ ]:
# Set the path for our PDF on E: drive
pdf_path = "E:\\l&w\\Policy.pdf"

In [ ]:
# Read the PDF and extract its text
reader = PyPDF2.PdfReader(pdf_path)
text = ""
for page in reader.pages:
    text += page.extract_text()

# Show the extracted text
print("Extracted Text:", text)

In [ ]:
#!pip install faiss-cpu

In [ ]:
#!conda install -c conda-forge faiss-cpu

In [ ]:
# Split the text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_text(text)

# Turn the chunks into numbers (embeddings) using LLaMA
#embeddings = OllamaEmbeddings(model="llama3.1")
embeddings = OllamaEmbeddings(model="mistral")

# Store the chunks and their embeddings in FAISS for searching
vector_store = FAISS.from_texts(documents, embeddings)

# Show how many chunks we created
print("Number of Chunks:", len(documents))

In [ ]:
# Setup Mistral as our AI model
llm = Ollama(model="mistral")

# Build the RAG system
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [ ]:
# Test the RAG system with a question
query = "What is the interest rate for a loan?"
response = qa_chain.run(query)
print("RAG Response:", response)

In [ ]:
# Test the RAG system with a question
query = "what is the criteria to take more than ten thousand in loan?"
response = qa_chain.run(query)
print("RAG Response:", response)

In [ ]:
pdf_path1 = "E:\\l&w\\Policy1.pdf"

In [ ]:
# Read the updated banking PDF
reader = PyPDF2.PdfReader(pdf_path1)
text = ""
for page in reader.pages:
    text += page.extract_text()
print("Updated Banking Text:", text)

## Poker Assistant Using RAG

In [ ]:
# Create a PDF with poker rules
poker_pdf_path = "E:\\l&w\\poker_rules.pdf"

In [ ]:
# Read the poker rules PDF
poker_reader = PyPDF2.PdfReader(poker_pdf_path)
poker_text = ""
for page in poker_reader.pages:
    poker_text += page.extract_text()

In [ ]:
# Split and embed the poker rules
poker_documents = text_splitter.split_text(poker_text)
poker_vector_store = FAISS.from_texts(poker_documents, embeddings)

# Setup RAG for the poker assistant
poker_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=poker_vector_store.as_retriever()
)

In [ ]:
# Test the poker assistant
game_query = "I have a Flush, should I bet or fold?"
poker_response = poker_qa_chain.run(game_query)
print("Poker Assistant Response:", poker_response)

In [ ]:
# Setup Mistral as our AI model
llm = Ollama(model="mistral")

# Create a custom prompt for better answers
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Use the following context to answer the question in a concise and specific way:\nContext: {context}\nQuestion: {question}\nAnswer:"
)
# Build the RAG system for banking with improved retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 4}),  # Fetch top 4 chunks
    chain_type_kwargs={"prompt": prompt_template}
)

# Test with varied banking questions
queries = [
    "What is the interest rate for a loan if my credit score is 800?",
    "What do I need for a $15,000 loan?",
    "What happens if I make a $60,000 transaction?"
]
for query in queries:
    response = qa_chain.run(query)
    print(f"Question: {query}\nAnswer: {response}\n")

In [ ]:
# Build the RAG system for poker
poker_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=poker_vector_store.as_retriever(search_kwargs={"k": 4}),
    chain_type_kwargs={"prompt": prompt_template}
)

# Test with varied poker questions
poker_queries = [
    "I have a Flush, should I bet or fold?",
    "Should I bluff with a High Card?",
    "What should I do in a late position with One Pair?"
]
for query in poker_queries:
    poker_response = poker_qa_chain.run(query)
    print(f"Question: {query}\nAnswer: {poker_response}\n")

## Agents

In [ ]:
!ollama serve

In [ ]:
# Install required libraries for LangGraph, CrewAI, and AutoGen
#!pip install langgraph crewai autogen

# Ensure Ollama is running (run in a terminal: ollama serve)
# Pull LLaMA 3.1 (4.1 GB)
# ollama pull llama3.1

# Verify Mistral and LLaMA are available
!ollama list

# Import basic libraries
import os

## LangGraph - Building Agent Workflows

In [ ]:
#!pip install graphviz

In [ ]:
#dot -V

In [ ]:
#setx PATH "%PATH%;C:\Program Files\Graphviz\bin"

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_community.llms import Ollama
from typing import TypedDict, Annotated
import pandas as pd
from graphviz import Digraph
from IPython.display import display
# Define the state (data that agents share)
class GameState(TypedDict):
    raw_data: str
    summary: str

In [ ]:
# Create sample poker game data
data = pd.DataFrame({
    "player": ["P1", "P2", "P3"],
    "action": ["bet 1000", "fold", "raise 5000"],
    "hand": ["Flush", "Pair", "Straight"]
})
data_path = "poker_logs.csv"
data.to_csv(data_path, index=False)

In [ ]:
# Agent 1: Load and process data
def process_data(state: GameState) -> GameState:
    df = pd.read_csv(data_path)
    state["raw_data"] = df.to_string()
    return state

In [ ]:
# Agent 2: Summarize data using Mistral
def summarize_data(state: GameState) -> GameState:
    llm = Ollama(model="mistral")
    prompt = f"Summarize this poker game data:\n{state['raw_data']}"
    state["summary"] = llm.invoke(prompt)
    return state

In [ ]:
# Build the workflow
workflow = StateGraph(GameState)
workflow.add_node("process_data", process_data)
workflow.add_node("summarize_data", summarize_data)
workflow.add_edge("process_data", "summarize_data")
workflow.add_edge("summarize_data", END)
workflow.set_entry_point("process_data")

In [ ]:
# Visualize LangGraph workflow
def visualize_workflow(workflow):
    dot = Digraph(comment="LangGraph Workflow")
    dot.attr(rankdir="LR")  # Left to right layout

    # Add nodes
    for node in workflow.nodes:
        dot.node(node, node)

    # Add entry point (safely access internal attribute)
    dot.node("START", "START", shape="circle")

    # Use protected member to get entry point (LangGraph stores it here)
    entry_point = workflow._entry_point if hasattr(workflow, "_entry_point") else list(workflow.nodes)[0]
    dot.edge("START", entry_point)

    # Add edges
    for edge in workflow.edges:
        dot.edge(edge[0], edge[1])

    # Add END node
    dot.node("END", "END", shape="doublecircle")
    return dot

In [ ]:
# Display the graph
graph = visualize_workflow(workflow)
display(graph)

In [ ]:
# Run the workflow
app = workflow.compile()
result = app.invoke({"raw_data": "", "summary": ""})
print("LangGraph Result - Summary:", result["summary"])

## CrewAI - Multi-Agent Collaboration

In [ ]:
from crewai import Agent
from langchain_ollama import OllamaLLM

In [ ]:
!ollama list

In [ ]:
from crewai import Agent
from langchain_ollama import OllamaLLM  # ✅ Correct new import

# Define agents
researcher = Agent(
    role="Poker Researcher",
    goal="Research poker hand rankings",
    backstory="Expert in poker rules",
    llm=OllamaLLM(model="mistral")  # ✅ Matches your installed model
)

strategist = Agent(
    role="Poker Strategist",
    goal="Suggest a betting strategy",
    backstory="Experienced poker player",
    llm=OllamaLLM(model="llama3.1")  # ✅ Use the correct local model name
)


In [ ]:
#!pip install langchain-ollama --upgrade

In [ ]:
llm = OllamaLLM(model="llama3.1")
print(llm.invoke("What is the best starting poker hand?"))

In [ ]:
from langchain_ollama import OllamaLLM

In [ ]:
pip show pydantic

In [ ]:
# Define tasks
research_task = Task(
    description="List poker hand rankings from highest to lowest",
    agent=researcher,
    expected_output="A list of poker hand rankings"
)

strategy_task = Task(
    description="Suggest a betting strategy for a player with a Flush",
    agent=strategist,
    expected_output="A betting strategy suggestion"
)

In [ ]:
# Create the crew
crew = Crew(
    agents=[researcher, strategist],
    tasks=[research_task, strategy_task],
    verbose=True
)

In [ ]:
# Run the crew
result = crew.kickoff()
print("CrewAI Result:", result)

## AutoGen - Agent Communication

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

In [ ]:
#from autogen import Assistant, UserProxyAgent, config_list_from_json

# Configure LLaMA via Ollama
config_list = {"model": "llama3.1","api_type": "ollama","base_url": "http://localhost:11434"}

In [ ]:
# Define agents
user_proxy = UserProxyAgent(
    name="Player",
    #human_in_the_loop=False,
    llm_config=config_list
)

analyst = AssistantAgent(
    name="Analyst",
    system_message="You are a poker analyst. Discuss game scenarios and suggest moves.",
    llm_config=config_list
)

In [ ]:
# Start a conversation
user_proxy.initiate_chat(
    analyst,
    message="I have a Straight, and my opponent raised 5000. Should I call or fold?"
)

In [ ]:
#!pip install ag2[ollama]

## Multi-Agent Project: Poker Game Analysis and Strategy System

In [ ]:
#!pip install langgraph
!pip install langchain-ollama

In [1]:
# Import libraries
from langgraph.graph import StateGraph, END
#from crewai import Agent, Task, Crew
#from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from langchain_community.llms import Ollama
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from typing import TypedDict, Annotated
import PyPDF2

ModuleNotFoundError: No module named 'langchain_core.memory'

In [ ]:
#!pip install langchain-core

In [ ]:
# Step 1: Create and save poker game data
data = pd.DataFrame({
    "player": ["P1", "P2", "P3", "P4"],
    "action": ["bet 1000", "fold", "raise 5000", "call"],
    "hand": ["Flush", "Pair", "Straight", "Two Pair"],
    "win": [1, 0, 1, 0]  # 1 = win, 0 = lose
})
data_path = "E://l&w/poker_logs.csv"
data.to_csv(data_path, index=False)

In [ ]:
pdf_path = "E:\\l&w\\poker_hands.pdf"

In [ ]:
data

In [ ]:
# Step 3: Setup RAG for hand rankings (using Mistral)
reader = PyPDF2.PdfReader(pdf_path)
text = ""
for page in reader.pages:
    text += page.extract_text()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
documents = text_splitter.split_text(text)
embeddings = OllamaEmbeddings(model="mistral")
vector_store = FAISS.from_texts(documents, embeddings)
llm_mistral = Ollama(model="mistral")
qa_chain = RetrievalQA.from_chain_type(llm=llm_mistral, chain_type="stuff", retriever=vector_store.as_retriever())

In [ ]:
# Step 4: Define the state for LangGraph
class PokerState(TypedDict):
    data: str
    win_prob: float
    hand_rank: str
    strategy: str

In [ ]:
# Agent 1: Data Analyst (processes game data, uses Mistral)
def analyze_data(state: PokerState) -> PokerState:
    df = pd.read_csv(data_path)
    llm = Ollama(model="mistral")
    state["data"] = llm.invoke(f"Summarize this poker game data:\n{df.to_string()}")
    return state

In [ ]:
# Agent 2: ML Predictor (predicts win probability)
def predict_win(state: PokerState) -> PokerState:
    df = pd.read_csv(data_path)
    X = pd.get_dummies(df[["action", "hand"]])
    y = df["win"]
    model = RandomForestClassifier(n_estimators=10, random_state=42)
    model.fit(X, y)
    sample = pd.get_dummies(pd.DataFrame({"action": ["raise 5000"], "hand": ["Straight"]}))
    sample = sample.reindex(columns=X.columns, fill_value=0)
    state["win_prob"] = model.predict_proba(sample)[0][1]
    return state

In [ ]:
# Agent 3: Hand Ranker (uses RAG with Mistral)
def rank_hand(state: PokerState) -> PokerState:
    query = "What is the rank of a Straight compared to a Flush?"
    state["hand_rank"] = qa_chain.run(query)
    return state

In [ ]:
# Agent 4: Suggest Strategy (using LangGraph, not CrewAI)
def suggest_strategy(state: PokerState) -> PokerState:
    llm = Ollama(model="llama3.1")  # LLaMA for reasoning
    strategy_prompt = f"Player has a Straight, win probability is {state['win_prob']:.2f}. Suggest a strategy."
    state["strategy"] = llm.invoke(strategy_prompt)
    return state

In [ ]:
# Agent 5: Strategy Review (using LangGraph, not CrewAI)
def review_strategy(state: PokerState) -> PokerState:
    llm = Ollama(model="llama3.1")
    review_prompt = f"Suggested strategy: {state['strategy']}. Win probability: {state['win_prob']:.2f}. Refine and review this strategy."
    state["strategy"] = llm.invoke(review_prompt)
    return state

# Build the LangGraph workflow
workflow = StateGraph(PokerState)
workflow.add_node("analyze_data", analyze_data)
workflow.add_node("predict_win", predict_win)
workflow.add_node("rank_hand", rank_hand)
workflow.add_node("suggest_strategy", suggest_strategy)
workflow.add_node("review_strategy", review_strategy)
workflow.add_edge("analyze_data", "predict_win")
workflow.add_edge("predict_win", "rank_hand")
workflow.add_edge("rank_hand", "suggest_strategy")
workflow.add_edge("suggest_strategy", "review_strategy")
workflow.add_edge("review_strategy", END)
workflow.set_entry_point("analyze_data")

# Run the workflow
app = workflow.compile()
result = app.invoke({"data": "", "win_prob": 0.0, "hand_rank": "", "strategy": ""})

print("Project Result:", result)

In [ ]:
'''# Agent 4: Strategy Crew (CrewAI team to suggest strategy, uses LLaMA)
def suggest_strategy(state: PokerState) -> PokerState:
    strategist = Agent(
        role="Poker Strategist",
        goal="Suggest a betting strategy",
        backstory="Experienced poker player",
        llm=Ollama(model="llama3.1")  # LLaMA for reasoning
    )
    strategy_task = Task(
        description=f"Player has a Straight, win probability is {state['win_prob']:.2f}. Suggest a strategy.",
        agent=strategist,
        expected_output="A betting strategy suggestion"
    )
    crew = Crew(agents=[strategist], tasks=[strategy_task], verbose=True)
    state["strategy"] = crew.kickoff()
    return state'''

In [ ]:
'''# Agent 5: AutoGen Discussion (agents discuss the strategy, uses LLaMA)
def discuss_strategy(state: PokerState) -> PokerState:
    config_list = {"model": "llama3.1", "api_type": "ollama", "base_url": "http://localhost:11434"}
    user_proxy = UserProxyAgent(name="Player", human_in_the_loop=False, llm_config=config_list)
    reviewer = AssistantAgent(
        name="Reviewer",
        system_message="Review and refine poker strategies.",
        llm_config=config_list
    )
    user_proxy.initiate_chat(
        reviewer,
        message=f"Suggested strategy: {state['strategy']}. Win probability: {state['win_prob']:.2f}. Refine this strategy."
    )
    return state'''

In [ ]:
# Build the LangGraph workflow
workflow = StateGraph(PokerState)
workflow.add_node("analyze_data", analyze_data)
workflow.add_node("predict_win", predict_win)
workflow.add_node("rank_hand", rank_hand)
workflow.add_node("suggest_strategy", suggest_strategy)
workflow.add_node("discuss_strategy", discuss_strategy)
workflow.add_edge("analyze_data", "predict_win")
workflow.add_edge("predict_win", "rank_hand")
workflow.add_edge("rank_hand", "suggest_strategy")
workflow.add_edge("suggest_strategy", "discuss_strategy")
workflow.add_edge("discuss_strategy", END)
workflow.set_entry_point("analyze_data")

In [ ]:
# Run the workflow
app = workflow.compile()
result = app.invoke({"data": "", "win_prob": 0.0, "hand_rank": "", "strategy": ""})
print("Project Result:", result)

In [ ]:
## App

In [ ]:
# Install Streamlit and other dependencies
!pip install streamlit langgraph crewai autogen scikit-learn
!conda install -c conda-forge langchain langchain-community pypdf2 faiss-cpu -y
!pip install -U langchain-ollama

# Ensure Ollama is running (run in a terminal: ollama serve)
# Verify Mistral and LLaMA are available
!ollama list

In [ ]:
import streamlit as st
from langgraph.graph import StateGraph, END
from crewai import Agent, Task, Crew
from autogen import Assistant, UserProxyAgent, config_list_from_json
from langchain_community.llms import Ollama
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from typing import TypedDict, Annotated
import PyPDF2
import os

# Streamlit app title and description
st.title("Poker Analysis Assistant")
st.write("Input your poker game details to get AI-driven insights, predictions, and strategies.")

# User inputs
player_hand = st.selectbox("Your Hand", ["Flush", "Straight", "Two Pair", "Pair", "High Card"])
opponent_action = st.selectbox("Opponent's Action", ["bet 1000", "raise 5000", "fold", "call"])
if st.button("Analyze"):

    # Step 1: Create and save poker game data
    data = pd.DataFrame({
        "player": ["P1", "P2", "P3", "P4"],
        "action": ["bet 1000", "fold", "raise 5000", "call"],
        "hand": ["Flush", "Pair", "Straight", "Two Pair"],
        "win": [1, 0, 1, 0]
    })
    data_path = "E:\\AI_Training\\poker_logs.csv"
    data.to_csv(data_path, index=False)

    # Step 2: Create a PDF with poker hand rankings for RAG
    pdf_path = "E:\\AI_Training\\poker_hands.pdf"
    with open(pdf_path, "w") as f:
        f.write("""
        Poker Hand Rankings
        1. Royal Flush: Ace, King, Queen, Jack, 10, all same suit.
        2. Straight Flush: Five consecutive cards, same suit.
        3. Four of a Kind: Four cards of the same rank.
        4. Full House: Three of a kind and a pair.
        5. Flush: Five cards of the same suit.
        6. Straight: Five consecutive cards, different suits.
        7. Three of a Kind: Three cards of the same rank.
        8. Two Pair: Two different pairs.
        9. One Pair: Two cards of the same rank.
        10. High Card: Highest card wins if no other hand is made.
        """)

    # Step 3: Setup RAG for hand rankings (using Mistral)
    reader = PyPDF2.PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    documents = text_splitter.split_text(text)
    embeddings = OllamaEmbeddings(model="mistral")
    vector_store = FAISS.from_texts(documents, embeddings)
    llm_mistral = Ollama(model="mistral")
    qa_chain = RetrievalQA.from_chain_type(llm=llm_mistral, chain_type="stuff", retriever=vector_store.as_retriever())

    # Step 4: Define the state for LangGraph
    class PokerState(TypedDict):
        data: str
        win_prob: float
        hand_rank: str
        strategy: str

    # Agent 1: Data Analyst (processes game data, uses Mistral)
    def analyze_data(state: PokerState) -> PokerState:
        df = pd.read_csv(data_path)
        llm = Ollama(model="mistral")
        state["data"] = llm.invoke(f"Summarize this poker game data:\n{df.to_string()}")
        return state

    # Agent 2: ML Predictor (predicts win probability)
    def predict_win(state: PokerState) -> PokerState:
        df = pd.read_csv(data_path)
        X = pd.get_dummies(df[["action", "hand"]])
        y = df["win"]
        model = RandomForestClassifier(n_estimators=10, random_state=42)
        model.fit(X, y)
        sample = pd.get_dummies(pd.DataFrame({"action": [opponent_action], "hand": [player_hand]}))
        sample = sample.reindex(columns=X.columns, fill_value=0)
        state["win_prob"] = model.predict_proba(sample)[0][1]
        return state

    # Agent 3: Hand Ranker (uses RAG with Mistral)
    def rank_hand(state: PokerState) -> PokerState:
        query = f"What is the rank of a {player_hand} compared to a Flush?"
        state["hand_rank"] = qa_chain.run(query)
        return state

    # Agent 4: Strategy Crew (CrewAI team to suggest strategy, uses LLaMA)
    def suggest_strategy(state: PokerState) -> PokerState:
        strategist = Agent(
            role="Poker Strategist",
            goal="Suggest a betting strategy",
            backstory="Experienced poker player",
            llm=Ollama(model="llama3.1")
        )
        strategy_task = Task(
            description=f"Player has a {player_hand}, win probability is {state['win_prob']:.2f}. Suggest a strategy.",
            agent=strategist,
            expected_output="A betting strategy suggestion"
        )
        crew = Crew(agents=[strategist], tasks=[strategy_task], verbose=True)
        state["strategy"] = crew.kickoff()
        return state

    # Agent 5: AutoGen Discussion (agents discuss the strategy, uses LLaMA)
    def discuss_strategy(state: PokerState) -> PokerState:
        config_list = [{"model": "llama3.1", "api_type": "ollama", "base_url": "http://localhost:11434"}]
        user_proxy = UserProxyAgent(name="Player", human_in_the_loop=False, llm_config=config_list)
        reviewer = Assistant(
            name="Reviewer",
            system_message="Review and refine poker strategies.",
            llm_config=config_list
        )
        user_proxy.initiate_chat(
            reviewer,
            message=f"Suggested strategy: {state['strategy']}. Win probability: {state['win_prob']:.2f}. Refine this strategy."
        )
        return state

    # Build the LangGraph workflow
    workflow = StateGraph(PokerState)
    workflow.add_node("analyze_data", analyze_data)
    workflow.add_node("predict_win", predict_win)
    workflow.add_node("rank_hand", rank_hand)
    workflow.add_node("suggest_strategy", suggest_strategy)
    workflow.add_node("discuss_strategy", discuss_strategy)
    workflow.add_edge("analyze_data", "predict_win")
    workflow.add_edge("predict_win", "rank_hand")
    workflow.add_edge("rank_hand", "suggest_strategy")
    workflow.add_edge("suggest_strategy", "discuss_strategy")
    workflow.add_edge("discuss_strategy", END)
    workflow.set_entry_point("analyze_data")

    # Run the workflow
    app = workflow.compile()
    result = app.invoke({"data": "", "win_prob": 0.0, "hand_rank": "", "strategy": ""})

    # Display results
    st.subheader("Analysis Results")
    st.write("**Game Data Summary:**", result["data"])
    st.write("**Win Probability:**", f"{result['win_prob']:.2f}")
    st.write("**Hand Rank Comparison:**", result["hand_rank"])
    st.write("**Suggested Strategy:**", result["strategy"])

In [ ]:
## tools and functions

In [ ]:
import streamlit as st
from langgraph.graph import StateGraph, END
from crewai import Agent, Task, Crew
from autogen import Assistant, UserProxyAgent, config_list_from_json
from langchain_community.llms import Ollama
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from typing import TypedDict, Annotated
import PyPDF2
import os
import requests

# Streamlit app title and description
st.title("Poker Analysis Assistant")
st.write("Input your poker game details to get AI-driven insights, predictions, and strategies, enhanced with tools and web APIs.")

# User inputs
player_hand = st.selectbox("Your Hand", ["Flush", "Straight", "Two Pair", "Pair", "High Card"])
opponent_action = st.selectbox("Opponent's Action", ["bet 1000", "raise 5000", "fold", "call"])
if st.button("Analyze"):

    # Step 1: Create and save poker game data
    data = pd.DataFrame({
        "player": ["P1", "P2", "P3", "P4"],
        "action": ["bet 1000", "fold", "raise 5000", "call"],
        "hand": ["Flush", "Pair", "Straight", "Two Pair"],
        "win": [1, 0, 1, 0]
    })
    data_path = "E:\\AI_Training\\poker_logs.csv"
    data.to_csv(data_path, index=False)

    # Step 2: Create a PDF with poker hand rankings for RAG
    pdf_path = "E:\\AI_Training\\poker_hands.pdf"
    with open(pdf_path, "w") as f:
        f.write("""
        Poker Hand Rankings
        1. Royal Flush: Ace, King, Queen, Jack, 10, all same suit.
        2. Straight Flush: Five consecutive cards, same suit.
        3. Four of a Kind: Four cards of the same rank.
        4. Full House: Three of a kind and a pair.
        5. Flush: Five cards of the same suit.
        6. Straight: Five consecutive cards, different suits.
        7. Three of a Kind: Three cards of the same rank.
        8. Two Pair: Two different pairs.
        9. One Pair: Two cards of the same rank.
        10. High Card: Highest card wins if no other hand is made.
        """)

    # Step 3: Setup RAG for hand rankings (using Mistral)
    reader = PyPDF2.PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    documents = text_splitter.split_text(text)
    embeddings = OllamaEmbeddings(model="mistral")
    vector_store = FAISS.from_texts(documents, embeddings)
    llm_mistral = Ollama(model="mistral")
    qa_chain = RetrievalQA.from_chain_type(llm=llm_mistral, chain_type="stuff", retriever=vector_store.as_retriever())

    # Step 4: Define the state for LangGraph
    class PokerState(TypedDict):
        data: str
        win_prob: float
        hand_rank: str
        strategy: str
        odds: float
        historical_win_rate: float

    # Custom Function: Calculate poker odds
    def calculate_odds(hand: str) -> float:
        odds_dict = {
            "Flush": 0.75,
            "Straight": 0.65,
            "Two Pair": 0.50,
            "Pair": 0.30,
            "High Card": 0.10
        }
        return odds_dict.get(hand, 0.10)

    # Mock Web API: Fetch historical win rate for a hand
    def fetch_historical_win_rate(hand: str) -> float:
        # Simulate an API call (in real-world, use requests.get() with a poker stats API)
        # Mock response: Historical win rates for hands
        win_rate_dict = {
            "Flush": 0.82,
            "Straight": 0.70,
            "Two Pair": 0.55,
            "Pair": 0.35,
            "High Card": 0.15
        }
        return win_rate_dict.get(hand, 0.15)

    # Agent 1: Data Analyst (processes game data, uses Mistral)
    def analyze_data(state: PokerState) -> PokerState:
        df = pd.read_csv(data_path)
        llm = Ollama(model="mistral")
        state["data"] = llm.invoke(f"Summarize this poker game data:\n{df.to_string()}")
        return state

    # Agent 2: ML Predictor (predicts win probability)
    def predict_win(state: PokerState) -> PokerState:
        df = pd.read_csv(data_path)
        X = pd.get_dummies(df[["action", "hand"]])
        y = df["win"]
        model = RandomForestClassifier(n_estimators=10, random_state=42)
        model.fit(X, y)
        sample = pd.get_dummies(pd.DataFrame({"action": [opponent_action], "hand": [player_hand]}))
        sample = sample.reindex(columns=X.columns, fill_value=0)
        state["win_prob"] = model.predict_proba(sample)[0][1]
        return state

    # Agent 3: Hand Ranker (uses RAG with Mistral)
    def rank_hand(state: PokerState) -> PokerState:
        query = f"What is the rank of a {player_hand} compared to a Flush?"
        state["hand_rank"] = qa_chain.run(query)
        return state

    # Agent 4: Odds Calculator (uses custom function and web API)
    def calculate_odds_and_stats(state: PokerState) -> PokerState:
        # Use custom function to calculate odds
        state["odds"] = calculate_odds(player_hand)
        # Use mock web API to fetch historical win rate
        state["historical_win_rate"] = fetch_historical_win_rate(player_hand)
        return state

    # Agent 5: Strategy Crew (CrewAI team to suggest strategy, uses LLaMA)
    def suggest_strategy(state: PokerState) -> PokerState:
        strategist = Agent(
            role="Poker Strategist",
            goal="Suggest a betting strategy",
            backstory="Experienced poker player",
            llm=Ollama(model="llama3.1")
        )
        strategy_task = Task(
            description=f"Player has a {player_hand}, win probability is {state['win_prob']:.2f}, odds are {state['odds']:.2f}, historical win rate is {state['historical_win_rate']:.2f}. Suggest a strategy.",
            agent=strategist,
            expected_output="A betting strategy suggestion"
        )
        crew = Crew(agents=[strategist], tasks=[strategy_task], verbose=True)
        state["strategy"] = crew.kickoff()
        return state

    # Agent 6: AutoGen Discussion (agents discuss the strategy, uses LLaMA)
    def discuss_strategy(state: PokerState) -> PokerState:
        config_list = [{"model": "llama3.1", "api_type": "ollama", "base_url": "http://localhost:11434"}]
        user_proxy = UserProxyAgent(name="Player", human_in_the_loop=False, llm_config=config_list)
        reviewer = Assistant(
            name="Reviewer",
            system_message="Review and refine poker strategies.",
            llm_config=config_list
        )
        user_proxy.initiate_chat(
            reviewer,
            message=f"Suggested strategy: {state['strategy']}. Win probability: {state['win_prob']:.2f}. Odds: {state['odds']:.2f}. Historical win rate: {state['historical_win_rate']:.2f}. Refine this strategy."
        )
        return state

    # Build the LangGraph workflow
    workflow = StateGraph(PokerState)
    workflow.add_node("analyze_data", analyze_data)
    workflow.add_node("predict_win", predict_win)
    workflow.add_node("rank_hand", rank_hand)
    workflow.add_node("calculate_odds_and_stats", calculate_odds_and_stats)
    workflow.add_node("suggest_strategy", suggest_strategy)
    workflow.add_node("discuss_strategy", discuss_strategy)
    workflow.add_edge("analyze_data", "predict_win")
    workflow.add_edge("predict_win", "rank_hand")
    workflow.add_edge("rank_hand", "calculate_odds_and_stats")
    workflow.add_edge("calculate_odds_and_stats", "suggest_strategy")
    workflow.add_edge("suggest_strategy", "discuss_strategy")
    workflow.add_edge("discuss_strategy", END)
    workflow.set_entry_point("analyze_data")

    # Run the workflow
    app = workflow.compile()
    result = app.invoke({"data": "", "win_prob": 0.0, "hand_rank": "", "strategy": "", "odds": 0.0, "historical_win_rate": 0.0})

    # Display results
    st.subheader("Analysis Results")
    st.write("**Game Data Summary:**", result["data"])
    st.write("**Win Probability (ML Prediction):**", f"{result['win_prob']:.2f}")
    st.write("**Hand Rank Comparison (RAG):**", result["hand_rank"])
    st.write("**Calculated Odds (Custom Function):**", f"{result['odds']:.2f}")
    st.write("**Historical Win Rate (Web API):**", f"{result['historical_win_rate']:.2f}")
    st.write("**Suggested Strategy (Agentic AI):**", result["strategy"])

In [ ]:
https://openweathermap.org/city/2643743

In [ ]:
import streamlit as st
import requests

# Streamlit app title and description
st.title("Weather-Based Poker Recommendation")
st.write("Check if today’s weather in LA is good for a poker game!")

# OpenWeatherMap API setup
API_KEY = "YOUR_API_KEY_HERE"  # Replace with your OpenWeatherMap API key
CITY = "Los Angeles"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

# Function to fetch weather data
def get_weather(city):
    params = {"q": city, "appid": API_KEY, "units": "metric"}  # Metric for Celsius
    try:
        response = requests.get(BASE_URL, params=params)
        data = response.json()
        if response.status_code == 200:
            temp = data["main"]["temp"]
            condition = data["weather"][0]["main"]
            return temp, condition
        else:
            return None, f"Error: {data['message']}"
    except Exception as e:
        return None, f"Error fetching weather: {str(e)}"

# Function to recommend poker game based on weather
def recommend_poker_game(temp, condition):
    if temp is None:
        return "Unable to make a recommendation due to weather fetch error."
    if temp > 30 or condition.lower() in ["rain", "thunderstorm"]:
        return "Weather is too hot or rainy. Consider playing poker indoors!"
    elif 15 <= temp <= 30 and condition.lower() in ["clear", "clouds"]:
        return "Weather is great! It’s a perfect day for a poker game outdoors!"
    else:
        return "Weather is okay. You can play poker, but indoor might be more comfortable."

# Fetch weather and make recommendation
if st.button("Check Weather and Recommendation"):
    temp, condition = get_weather(CITY)
    recommendation = recommend_poker_game(temp, condition)
    
    # Display results
    st.subheader("Weather in LA")
    if temp is not None:
        st.write(f"Temperature: {temp}°C")
        st.write(f"Condition: {condition}")
    else:
        st.write(condition)  # Error message
    st.subheader("Recommendation")
    st.write(recommendation)

In [ ]:
dot -V

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_community.llms import Ollama
from typing import TypedDict, Annotated
import pandas as pd
from graphviz import Digraph
from IPython.display import display

# Define the state (data that agents share)
class GameState(TypedDict):
    raw_data: str
    summary: str

# Create sample poker game data
data = pd.DataFrame({
    "player": ["P1", "P2", "P3"],
    "action": ["bet 1000", "fold", "raise 5000"],
    "hand": ["Flush", "Pair", "Straight"]
})
data_path = "E:\\AI_Training\\poker_logs.csv"
data.to_csv(data_path, index=False)

# Agent 1: Load and process data
def process_data(state: GameState) -> GameState:
    df = pd.read_csv(data_path)
    state["raw_data"] = df.to_string()
    return state

# Agent 2: Summarize data using Mistral
def summarize_data(state: GameState) -> GameState:
    llm = Ollama(model="mistral")
    prompt = f"Summarize this poker game data:\n{state['raw_data']}"
    state["summary"] = llm.invoke(prompt)
    return state

# Build the workflow
workflow = StateGraph(GameState)
workflow.add_node("process_data", process_data)
workflow.add_node("summarize_data", summarize_data)
workflow.add_edge("process_data", "summarize_data")
workflow.add_edge("summarize_data", END)
workflow.set_entry_point("process_data")

# Visualize the workflow
def visualize_workflow(workflow):
    dot = Digraph(comment="LangGraph Workflow")
    dot.attr(rankdir="LR")  # Left to right layout

    # Add nodes
    for node in workflow.nodes:
        dot.node(node, node)

    # Add entry point
    dot.node("START", "START", shape="circle")
    dot.edge("START", workflow.entry_point)

    # Add edges
    for edge in workflow.edges:
        dot.edge(edge[0], edge[1])

    # Add END node
    dot.node("END", "END", shape="doublecircle")
    return dot

# Display the graph
graph = visualize_workflow(workflow)
display(graph)

# Run the workflow
app = workflow.compile()
result = app.invoke({"raw_data": "", "summary": ""})
print("LangGraph Result - Summary:", result["summary"])

In [ ]:
# Add this near the top of poker_app.py, after imports
from graphviz import Digraph

# Add this before "Run the workflow" (around line 185 in the previous poker_app.py)
# Visualize the workflow
def visualize_workflow(workflow):
    dot = Digraph(comment="LangGraph Workflow")
    dot.attr(rankdir="LR")

    # Add nodes
    for node in workflow.nodes:
        dot.node(node, node)

    # Add entry point
    dot.node("START", "START", shape="circle")
    dot.edge("START", workflow.entry_point)

    # Add edges
    for edge in workflow.edges:
        dot.edge(edge[0], edge[1])

    # Add END node
    dot.node("END", "END", shape="doublecircle")

    # Save the graph as a PNG
    dot.format = "png"
    dot.render("E:\\AI_Training\\workflow_graph", view=False)
    return "E:\\AI_Training\\workflow_graph.png"

# Add this after "Build the LangGraph workflow" (around line 183)
graph_path = visualize_workflow(workflow)

# Add this to the "Display results" section (around line 193)
st.subheader("Workflow Visualization")
st.image(graph_path, caption="LangGraph Workflow")